# Task 1: Binary Representations

Optional part: Calculate how many bits set in code, bits set is number of 1s

## 1. Implement Bitwise Left Rotation (`rotl`)

In [7]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotl(x, n=1):
    n %= 32  # Ensure n is within 0-31
    return ((x << n) & MASK_32) | ((x & MASK_32) >> (32 - n))

# Task 2

# Task 3

# Task 4

# Task 5

# Task 6

# Task 7

# Task 8